In [ ]:
import matplotlib.pyplot as plt
import pnstyler.styler as styler
from runner import load_data, main_path
import ml_confs
from pathlib import Path
import numpy as np
import os
import pickle

In [ ]:
configs = ml_confs.from_file(main_path / 'experiments/fluid/configs.yaml')
exp_path = Path(main_path / 'experiments/fluid')
results_path = exp_path / 'results/'
with np.load(exp_path / "data/cylinder2d_remesh.npz") as f:
    full_data = {k: np.asarray(v) for k, v in f.items()}

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = styler.figsize(width_to_height=3))

cmap = 'Spectral'
streamplot_args = {
    'color':(0, 0, 0, 0.2), #RGBA tuple for the color of the streamlines 0.5 alpha black 
    'density':1.5, 
    'linewidth':0.5, 
    'arrowsize':0.5, 
    'arrowstyle':'->'
}
ax_args = {
    'xticks': [],
    'yticks': [],
    'aspect': 'equal',
    'ymargin': 0
}

for ax_idx, t_idx in enumerate([50, 100]):
    X = full_data['x']
    Y = full_data['y']
    C = full_data['C'][..., t_idx] 
    U = full_data['U'][..., t_idx]
    V = full_data['V'][..., t_idx]
    ax = axes[ax_idx]
    contour = ax.contourf(X, Y, C, 200, cmap=cmap)
    field = ax.streamplot(X, Y, U, V, **streamplot_args)
    ax.set(**ax_args)
    # Plot a full black circle of radius 0.5
    theta = np.linspace(0, 2*np.pi, 100)
    r = 0.45
    ax.plot(r*np.cos(theta), r*np.sin(theta), 'k') 
    ax.fill_between(r*np.cos(theta), r*np.sin(theta), color='k')
fig.savefig('fluid_flow_dynamics.png', bbox_inches='tight', dpi=300)

In [ ]:
os.listdir(results_path)
_tmp = {}
for filename in os.listdir(results_path):
    model_name = filename.split('_')[0]
    if model_name not in _tmp:
        _tmp[model_name] = []
    with open(results_path / filename, 'rb') as f:
        restored = pickle.load(f)
    if restored is not None:
        _tmp[model_name].append(restored)

FLUID_FLOW = {}
_fluid_flow_std = 0.16754142
for model_name, model_data in _tmp.items():
    if len(model_data) > 0:
        FLUID_FLOW[model_name] = {
            'data': np.array(model_data).mean(axis=0)/_fluid_flow_std
        }

In [ ]:
FLUID_FLOW

In [ ]:
fig, ax = plt.subplots()
for model_name in FLUID_FLOW.keys():
    model_data = FLUID_FLOW[model_name]
    ax.plot(model_data['data'], label=model_name)
ax.legend()
ax.set_yscale('log')
